# Geospatial aggregates for poverty comparison

In support of the ELCPV project (PXXXXXX), GOST are calculating admin 2 level summaries of a series of risk, climate, and environmental data. The focus will be on the following datasets

| Data layer | Source | Link | Notes |
| :--- | :--- | :--- | :--- |
| Floods | Fathom | [DDH](https://datacatalog.worldbank.org/int/search/dataset/0021728/Global-Flood-Hazard--FATHOM-) | GFDRR are working on acquiring an updated dataset, need funding |
| Drought | **TBD**  |  | One of the candidates is utilizing TerraClimate's Precipitation, Potential Evapotranspiration and Runoff to derive the SPI and SPEI as proxy for Meteorological Drought and SRI for the Hydrological Drought for period 1958-2022 |
| Heatwave | DDP | [Github](https://github.com/datapartnership/heatwaves) | The Data Partnership have developed tools for evaluating heat waves, but it is a work in progress |
| Cyclone | [NOAA IBTrACS](https://www.ncei.noaa.gov/products/international-best-track-archive) | S3 bucket: [IBTrACS](s3://wbgdecinternal-ntl/climate/data/cyclone/ibtracs) | Available in points and lines shapefile, netCDF and CSV. From 25 Oct 1842 to 7 Sep 2023 |
| Wildfire | **TBD** |  | Global Fire Emissions Database [GFED](https://www.globalfiredata.org/) as one of potential data |
| Temperature | CCKP | [Climate Change Knowledge Portal](https://climateknowledgeportal.worldbank.org/download-data) | The CCKP are working on an updated, 1/4 degree resolution dataset that we will leverage |
| Precipitation | CCKP | [Climate Change Knowledge Portal](https://climateknowledgeportal.worldbank.org/download-data) | The CCKP are working on an updated, 1/4 degree resolution dataset that we will leverage |
| Sea-level rise | CCKP | CCKP's [3rd party site](https://data.cckp.adamplatform.eu/InundationMaps/) | The SLR projection available for all SSP's Scenario |
| Population | WorldPop | [WorldPop](https://www.worldpop.org/) | We hope to use the soon-to-be released Global WorldPop v2 |
| Schools | OSM |  | Extracted via [HOT Export Tool](https://export.hotosm.org/en/v3/) |
| Health facilities | OSM |  | [HOT Export Tool](https://export.hotosm.org/en/v3/) |
| Road density | OSM |  | [HOT Export Tool](https://export.hotosm.org/en/v3/) |
| Dams | **TBD** | [DDH](https://datacatalog.worldbank.org/search/dataset/0038478) | Global Dams Database available in DDH |

In [1]:
import sys, os, importlib, math, multiprocessing
import rasterio, geojson

import pandas as pd
import geopandas as gpd
import numpy as np

from h3 import h3
from tqdm import tqdm
from shapely.geometry import Polygon

sys.path.insert(0, "/home/wb411133/Code/gostrocks/src")
import GOSTRocks.rasterMisc as rMisc
import GOSTRocks.ntlMisc as ntl
import GOSTRocks.mapMisc as mapMisc
from GOSTRocks.misc import tPrint

sys.path.append("../../../src")
import h3_helper
import country_zonal

%load_ext autoreload
%autoreload 2

/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
# Extract data for mapping
sel_iso3 = 'ARG'
h3_lvl = 5

ntl_layers = ntl.aws_search_ntl()
global_urban = "/home/public/Data/GLOBAL/GHSL/SMOD/GHS_SMOD_E2020_GLOBE_R2023A_54009_1000_V1_0.tif"
global_population = "/home/public/Data/GLOBAL/Population/WorldPop_PPP_2020/WP_2020_100m.vrt"
admin_bounds = "/home/public/Data/GLOBAL/ADMIN/ADMIN2/HighRes_20230328/shp/WB_GAD_ADM2.shp"
out_folder = f"/home/wb411133/projects/Space2Stats/MAPPING/{sel_iso3}"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

In [3]:
all_countries = gpd.read_file(admin_bounds)
sel_country = all_countries.loc[all_countries['ISO_A3'] == sel_iso3]

In [8]:
len(all_countries.loc[all_countries['WB_REGION'] == 'LCR']['ISO_A3'].unique())

32

In [5]:
all_countries.head()

,ISO_A3,ISO_A2,WB_A3,HASC_0,HASC_1,HASC_2,GAUL_0,GAUL_1,GAUL_2,WB_REGION,...,NAM_1_WIKI,NAM_2,NAM_2_GAUL,NAM_2_STAT,NAM_2_SRCE,NAM_2_NTVE,NAM_2_WIKI,Shape_Leng,Shape_Area,geometry
0,AFG,AF,AFG,AF,AF.BD,AF.BD.AK,1,0,0,SAR,...,None,Arghanj Khwa,None,Arghanj Khwa,Arghanj Khwa,ارغنچخواه,None,1.617679,0.074255,"POLYGON ((70.77120 37.49522, 70.77751 37.50151..."
1,AFG,AF,AFG,AF,AF.BD,AF.BD.AR,1,0,0,SAR,...,None,Argo,None,Argo,Argo,ارگو,None,1.977449,0.106834,"POLYGON ((70.10776 37.01419, 70.10744 37.01628..."
2,AFG,AF,AFG,AF,AF.BD,AF.BD.BA,1,0,0,SAR,...,None,Baharak,None,Baharak,Baharak,بهارک,None,0.894333,0.032766,"POLYGON ((70.77153 37.05601, 70.77924 37.06589..."
3,AFG,AF,AFG,AF,AF.BD,AF.BD.DY,1,0,0,SAR,...,None,Darayem,None,Darayim,Darayem,درایم,None,1.295426,0.056675,"POLYGON ((70.65071 36.86746, 70.65227 36.86330..."
4,AFG,AF,AFG,AF,AF.BD,AF.BD.DA,1,0,0,SAR,...,None,Darwaz,None,Darwaz,Darwaz,درواز,None,1.888940,0.125982,"POLYGON ((70.99066 38.49031, 70.99164 38.48937..."


In [ ]:
sel_country['ISO_A3'].unique()

In [ ]:
# Clip nighttime lights
ntl_file = os.path.join(out_folder, "NTL_VIIRS.tif")
if not os.path.exists(ntl_file):
    rMisc.clipRaster(rasterio.open(ntl_layers[-1]), sel_country, ntl_file)

In [ ]:
# copy h3 cells
h3_file = f"s3://wbg-geography01/Space2Stats/h3_spatial_data/LCR/{sel_iso3}/h3_level_{h3_lvl}.geojson"
in_h3 = gpd.read_file(h3_file)
in_h3.to_file(os.path.join(out_folder, f'h3_level_{h3_lvl}.geojson'), driver='GeoJSON')

In [ ]:
# Copy admin boundaries
sel_country.to_file(os.path.join(out_folder, "adm2.geojson"), driver="GeoJSON")


# Read in configured data

In [ ]:
config_folder = "../../../data/config"
config_file = os.path.join(config_folder, 'chirps_rainfall_differenceanomaly.json')


In [ ]:
from space2stats_data_config import s2s_geo_data

In [ ]:
xx = s2s_geo_data(config_file)
xx.data_info

In [ ]:
xx.data_info['rainfall_differenceanomaly'].keys()

In [ ]:
# Define global inputs
admin_bounds = "/home/public/Data/GLOBAL/ADMIN/ADMIN2/HighRes_20230328/shp/WB_GAD_ADM2.shp"
global_urban_file = "/home/public/Data/GLOBAL/GHSL/SMOD/GHS_SMOD_E2020_GLOBE_R2023A_54009_1000_V1_0.tif"
global_pop_file = "/home/public/Data/GLOBAL/GHSL/Pop/GHS_POP_E2020_GLOBE_R2023A_54009_100_V1_0.tif"

In [ ]:
inA = gpd.read_file(admin_bounds)
inA['ID'] = inA.index #Create ID for indexing

In [ ]:
sel_iso3 = set(inA.loc[inA['WB_REGION'] == "LCR"]['WB_A3'].unique())
sel_iso3 = [x for x in sel_iso3 if x != None]
sorted(sel_iso3)

In [ ]:
arg_list = []
for iso3 in sel_iso3:
    selA = inA.loc[inA['WB_A3'] == iso3].copy()
    out_folder = f"/home/wb411133/projects/Space2Stats/COUNTRIES/{iso3}"
    print(out_folder)
    if not os.path.exists(out_folder):
        os.makedirs(out_folder)
    arg_list.append([iso3, selA, 6, global_pop_file, global_urban_file, out_folder])

In [ ]:
def multi_processing_single_raster(sel_iso3, inA, h3_level, r_file, urban_file, out_folder, zonal_label):
    out_file = os.path.join(out_folder, f'{sel_iso3}_{zonal_label}_{h3_level}.csv')
    print(out_file)
    if not os.path.exists(out_file):
        zonalC = country_zonal.country_h3_zonal(sel_iso3, selA, "ID", h3_level, out_folder)
        zonal_res_pop = zonalC.zonal_raster_urban(r_file, urban_file)
        zonal_res_pop.to_csv(out_file)
    return(out_file)

In [ ]:
multi_processing_single_raster(*arg_list[0], "GHS_POP_2020")

# DEBUGGING

In [ ]:
import boto3
bucket = 'wbgdecinternal-ntl'
prefix = 'climate/products/spei-terraclimate/zarr/'

s3client = boto3.client('s3')
objects = s3client.list_objects_v2(Bucket=bucket, Prefix=prefix)
for res in objects['Contents']:
    print(res['Key'])